In [1]:
import pandas as pd

In [5]:
data = pd.read_csv('tableau_project.csv')
data.head()

,Unnamed: 0,journey_id,vehicle_type_id,start_type,start_at,start_lat,start_lon,end_at,end_lat,end_lon,...,price_distance,price_duration,distance,duration,source,rider_waiting_time,driver_waiting_time,price_supplements,discount,rating
0,0,93e6d216088af74c32183283d4bb2953,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 11:08:59,19.431363,-99.195362,2017-12-16 11:37:32,19.406438,-99.175260,...,"5,204",0,"7,943","1,058",Android,590,48,662,0,9.0
1,1,444b084ec64244919afadfba879876a3,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 11:06:16,19.431567,-99.195368,2017-12-16 11:07:21,NaN,NaN,...,0,0,0,0,iPhone,NaN,NaN,0,0,9.0
2,2,dbcc57f4e7606a71c74c6e4666a977a4,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-11-24 22:58:37,19.422052,-99.174499,2017-11-24 23:27:05,19.438666,-99.180821,...,"3,552",0,"3,054",689,Android,923,4,0,0,10.0
3,3,1ffe9a24033847148c683574985df56c,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-12-16 09:17:08,19.433119,-99.154844,2017-12-16 09:17:30,19.290148,-99.144407,...,0,0,0,0,iPhone,NaN,NaN,0,0,9.0
4,4,fb1ef7d070724482a33be41f27737ea5,077866c3fd1a75f51ca7f8eae166ae32,asap,2017-11-24 18:11:00,19.440996,-99.183581,2017-11-24 18:31:31,19.422815,-99.174865,...,"2,974",12,"2,859",887,iPhone,329,36,0,0,8.0


In [8]:
data.shape

(3864, 22)

In [ ]:
data.describe()

TypeError: 'Series' object is not callable